In [68]:
import requests
requests.get("http://127.0.0.1:8000/").json()


{'Connection': 'Success'}

In [15]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from scipy import stats
from tempfile import mkdtemp
from shutil import rmtree


from sklearn import set_config
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.ensemble import AdaBoostRegressor, VotingRegressor, GradientBoostingRegressor, StackingRegressor, RandomForestRegressor
from sklearn.feature_selection import SelectPercentile, mutual_info_regression, VarianceThreshold, SelectFromModel
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.metrics import make_scorer, mean_squared_error, mean_squared_log_error
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder, StandardScaler, RobustScaler, MultiLabelBinarizer
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

In [63]:
data = pd.read_csv("../raw_data/X_y_data2.csv")
data = data[data.y < 2]
X = data.iloc[:, :-1]
y = data.iloc[:, -1:]

In [64]:
data

,Unnamed: 0,Organization Name,IPO Status,Number of Founders,Company Type,CB Rank (Company),Operating Status,Industry Groups,Number of Employees,Estimated Revenue Range,...,e_to_d_ratio,pe_to_e_ratio,total_funding_usd,last_funding_type,last_funding_amount,hub_tags,industry_multiples,company_valuation,months_since_founded,y
1,1,infinite roots,Private,1.0,For Profit,"1,875",Active,"Biotechnology, Food and Beverage, Health Care,...",51-100,NaN,...,0.0,0.0,86661948.0,Series B,57481860.0,NaN,3.0,2.599858e+08,77,1
2,2,instagrid,Private,2.0,For Profit,"1,950",Active,"Energy, Manufacturing",101-250,$1M to $10M,...,0.0,0.0,137401184.0,Series C,94927569.0,NaN,1.5,2.061018e+08,77,1
3,3,isar aerospace,Private,3.0,For Profit,"2,015",Active,"Information Technology, Science and Engineerin...",251-500,$10M to $50M,...,0.0,0.0,350641701.0,Series C,NaN,NaN,4.0,1.402567e+09,77,1
5,5,wandelbots,Private,8.0,For Profit,"2,128",Active,"Artificial Intelligence (AI), Consumer Electro...",101-250,$1M to $10M,...,0.0,0.0,122184409.0,Series C,84000000.0,NaN,4.0,4.887376e+08,80,1
6,6,agile robots ag,Private,3.0,For Profit,"2,553",Active,"Artificial Intelligence (AI), Data and Analyti...",501-1000,$50M to $100M,...,0.0,0.0,270050589.0,Venture - Series Unknown,30000000.0,Unicorn,4.0,1.080202e+09,77,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2170,2175,lofelt,Private,3.0,For Profit,"155,770",Active,"Consumer Electronics, Hardware",11-50,$1M to $10M,...,0.0,0.0,6745373.0,Series A,NaN,NaN,1.5,1.011806e+07,125,1
2177,2182,playtestcloud,Private,2.0,For Profit,"187,226",Active,"Data and Analytics, Design, Gaming, Mobile, Pl...",11-50,Less than $1M,...,0.0,0.0,129685.0,Grant,129684.0,NaN,4.0,5.187400e+05,121,1
2178,2183,ava,Private,2.0,For Profit,"188,048",Closed,"Data and Analytics, Government and Military, I...",11-50,$1M to $10M,...,0.0,0.0,5015405.0,Venture - Series Unknown,NaN,NaN,4.0,2.006162e+07,113,0
2181,2186,regional markt,Private,2.0,For Profit,"224,877",Closed,"Agriculture and Farming, Commerce and Shopping...",11-50,$1M to $10M,...,0.0,0.0,183876.0,Convertible Note,72797.0,NaN,2.0,3.677520e+05,125,0


In [99]:
data.funding_status.value_counts().index


Index(['M&A', 'Early Stage Venture', 'Seed', 'Late Stage Venture',
       'Private Equity', 'IPO'],
      dtype='object')

In [71]:
one_hot_category = [
    "funding_status", "industry_groups"
]

"""multi_hot_category = [
    "industry"
]"""

ordinal_category = [
    "no_employees", "revenue_range"
]

numerical_features = [
    'months_since_founded', 'website', 'phone', "no_founders", "lat", "lon",
    'email', 'linkedin', 'twitter', 'facebook', 'no_investors', 'no_fund_rounds',
    'no_sub_orgs', 'has_preseed', 'has_seed', 'has_series_a', 'has_series_b', 'has_series_c',
    'has_series_d', 'has_series_e', 'has_angel', 'has_debt_financing',
    'has_grant', 'has_corporate_round', 'has_series_x'
]

In [72]:
no_employees_ordinal = [
    '11-50', '51-100', '101-250', '251-500', '501-1000', '1001-5000', '5001-10000', '10001+'
]

In [73]:
revenue_range_ordinal = [
    'Less than $1M', '$1M to $10M', '$10M to $50M', '$50M to $100M', '$100M to $500M', '$500M to $1B', '$1B to $10B', '$10B+'
]

In [74]:
industry_multi = [
 'Biotechnology',
 'Energy',
 'Professional Services',
 'Government and Military',
 'Media and Entertainment',
 'Data and Analytics',
 'Consumer Electronics',
 'Apps',
 'Blockchain and Cryptocurrency',
 'Community and Lifestyle',
 'Software',
 'Gaming',
 'Consumer Goods',
 'Clothing and Apparel',
 'Artificial Intelligence (AI)',
 'Commerce and Shopping',
 'Health Care',
 'Sustainability',
 'Design',
 'Transportation',
 'Administrative Services',
 'Hardware',
 'Content and Publishing',
 'Information Technology',
 'Education',
 'Mobile',
 'Natural Resources',
 'Food and Beverage',
 'Travel and Tourism',
 'Events',
 'Agriculture and Farming',
 'Science and Engineering',
 'Other',
 'Privacy and Security',
 'Financial Services',
 'Manufacturing',
 'Sports',
 'Sales and Marketing',
 'Real Estate',
 'Advertising',
 'Internet Services',
 'Platforms',
 'Payments',
 'Lending and Investments',
 'Video',
 'Music and Audio',
 'Navigation and Mapping',
 'Social Impact',
 'Messaging and Telecommunications'
]

In [75]:
feat_ordinal_dict = {
    "no_employees": no_employees_ordinal,
    "revenue_range": revenue_range_ordinal
}

In [76]:
encoder_ordinal = OrdinalEncoder(
    categories = [feat_ordinal_dict[i] for i in ordinal_category],  
    dtype = np.int64
)


preproc_ordinal = make_pipeline(
    SimpleImputer(strategy = "most_frequent"),
    encoder_ordinal, 
    MinMaxScaler()
)

preproc_ordinal

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('ordinalencoder',
                 OrdinalEncoder(categories=[['11-50', '51-100', '101-250',
                                             '251-500', '501-1000', '1001-5000',
                                             '5001-10000', '10001+'],
                                            ['Less than $1M', '$1M to $10M',
                                             '$10M to $50M', '$50M to $100M',
                                             '$100M to $500M', '$500M to $1B',
                                             '$1B to $10B', '$10B+']],
                                dtype=<class 'numpy.int64'>)),
                ('minmaxscaler', MinMaxScaler())])

In [77]:
preproc_min_numerical = make_pipeline(
    KNNImputer(),
    MinMaxScaler())

preproc_min_numerical

Pipeline(steps=[('knnimputer', KNNImputer()), ('minmaxscaler', MinMaxScaler())])

In [78]:
preproc_nominal = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(handle_unknown="ignore")
)

preproc_nominal

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder', OneHotEncoder(handle_unknown='ignore'))])

In [79]:
preproc_multi = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    MultiLabelBinarizer(classes=industry_multi)
)

preproc_multi

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('multilabelbinarizer',
                 MultiLabelBinarizer(classes=['Biotechnology', 'Energy',
                                              'Professional Services',
                                              'Government and Military',
                                              'Media and Entertainment',
                                              'Data and Analytics',
                                              'Consumer Electronics', 'Apps',
                                              'Blockchain and Cryptocurrency',
                                              'Community and Lifestyle',
                                              'Software', 'Gaming',
                                              'Consumer Goods',
                                              'Clothing and Apparel',
                                              'Artificial Intelligence (AI)',
                                              'Commerce and Shopping',
                                              'Health Care', 'Sustainability',
                                              'Design', 'Transportation',
                                              'Administrative Services',
                                              'Hardware',
                                              'Content and Publishing',
                                              'Information Technology',
                                              'Education', 'Mobile',
                                              'Natural Resources',
                                              'Food and Beverage',
                                              'Travel and Tourism', 'Events', ...]))])

In [80]:
preproc_robust_numerical = make_pipeline(
    KNNImputer(),
    RobustScaler())

preproc_robust_numerical

Pipeline(steps=[('knnimputer', KNNImputer()), ('robustscaler', RobustScaler())])

In [81]:
preproc = make_column_transformer(
        (preproc_ordinal, ordinal_category),
        (preproc_min_numerical, numerical_features),
        (preproc_nominal, one_hot_category),
        # (preproc_multi, multi_hot_category),
        # (preproc_robust_numerical, robust_category),
        remainder="drop"
)

In [82]:
preproc

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['11-50',
                                                                              '51-100',
                                                                              '101-250',
                                                                              '251-500',
                                                                              '501-1000',
                                                                              '1001-5000',
                                                                              '5001-10000',
                                                                              '10001+'],
                                                                             ['Less '
                                                                              'than '
                                                                              '$1M',
                                                                              '$1M '
                                                                              'to '
                                                                              '$10M',
                                                                              '$10M '
                                                                              'to '
                                                                              '$50M',
                                                                              '$50M '
                                                                              'to '
                                                                              '$100M',
                                                                              '$100M '
                                                                              'to '
                                                                              '$500M',
                                                                              '$500M '
                                                                              'to '
                                                                              '...
                                  'no_sub_orgs', 'has_preseed', 'has_seed',
                                  'has_series_a', 'has_series_b',
                                  'has_series_c', 'has_series_d',
                                  'has_series_e', 'has_angel',
                                  'has_debt_financing', 'has_grant',
                                  'has_corporate_round', 'has_series_x']),
                                ('pipeline-3',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['funding_status', 'industry_groups'])])

In [83]:
X_preprocessed = pd.DataFrame(preproc.fit_transform(X, y))

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y.iloc[:, -1:], test_size=.20, random_state=1)


In [85]:
! pip install tensorflow

In [87]:
from tensorflow import keras
from keras import Model, Sequential, layers, regularizers, optimizers
from keras.callbacks import EarlyStopping

In [88]:
X_preprocessed.shape

(857, 52)

In [89]:
reg = regularizers.l1_l2(l2=0.005)

model = Sequential()
model.add(layers.Input(shape=(X_preprocessed.shape[1],)))
model.add(layers.Dense(50, activation="relu", kernel_regularizer=reg))
model.add(layers.BatchNormalization(momentum=0.9))
model.add(layers.Dropout(rate=0.1))
model.add(layers.Dense(20, activation="tanh"))
model.add(layers.BatchNormalization(momentum=0.9))
model.add(layers.Dropout(rate=0.1))
model.add(layers.Dense(1, activation="sigmoid"))

In [90]:
learning_rate = 0.01

optimizer = optimizers.Adam(learning_rate=learning_rate)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [91]:
es = EarlyStopping(
    patience=20,
    restore_best_weights=True,
    verbose=1
)

history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=700,
    batch_size=32,
    callbacks=[es],
    verbose=0
)

Restoring model weights from the end of the best epoch: 78.
Epoch 98: early stopping


In [92]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 50)                2650      
                                                                 
 batch_normalization_2 (Batc  (None, 50)               200       
 hNormalization)                                                 
                                                                 
 dropout_2 (Dropout)         (None, 50)                0         
                                                                 
 dense_4 (Dense)             (None, 20)                1020      
                                                                 
 batch_normalization_3 (Batc  (None, 20)               80        
 hNormalization)                                                 
                                                                 
 dropout_3 (Dropout)         (None, 20)               

In [93]:
history.history

{'loss': [4.209773540496826,
  3.7115042209625244,
  3.362804889678955,
  3.056623935699463,
  2.7470321655273438,
  2.493152141571045,
  2.2384846210479736,
  2.0367417335510254,
  1.8287433385849,
  1.6516040563583374,
  1.5077179670333862,
  1.3830960988998413,
  1.2663213014602661,
  1.1697044372558594,
  1.0842852592468262,
  1.0062416791915894,
  0.9358353614807129,
  0.8865991830825806,
  0.8466512560844421,
  0.8049336075782776,
  0.7583783268928528,
  0.7327357530593872,
  0.6935818791389465,
  0.684597909450531,
  0.6514851450920105,
  0.6269081234931946,
  0.6050158739089966,
  0.5824552178382874,
  0.5617969036102295,
  0.5447555184364319,
  0.527915358543396,
  0.5107505917549133,
  0.5255215764045715,
  0.4909460246562958,
  0.4640762507915497,
  0.48170432448387146,
  0.47975531220436096,
  0.4536363482475281,
  0.4317454695701599,
  0.4311845302581787,
  0.4208638370037079,
  0.439495325088501,
  0.4048273265361786,
  0.39781469106674194,
  0.3966769874095917,
  0.40976

In [23]:
df = pd.read_csv("../raw_data/X_y_data2.csv")
df = df[df.y > 1]
X_pred = df.iloc[:, :-2]

In [24]:
X_pred_preprocessed = preproc.transform(X_pred)

In [25]:
model.predict(X_pred_preprocessed)

42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array([[0.9611442 ],
       [0.9547018 ],
       [0.64973736],
       ...,
       [0.8771408 ],
       [0.4069949 ],
       [0.10386123]], dtype=float32)

In [26]:
df["y_pred"] = model.predict(X_pred_preprocessed)

42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [27]:
df

,Unnamed: 0,state,funding_status,revenue_range,no_employees,no_founders,industry_groups,website,phone,email,...,has_series_e,has_angel,has_debt_financing,has_grant,has_corporate_round,has_series_x,has_ico,total_funding_usd,y,y_pred
0,0,Bayern,Early Stage Venture,$100M to $500M,251-500,5.0,Retail and E-commerce,1,0,1,...,0,0,0,0,0,1,0,67922929.0,2,0.961144
4,4,Nordrhein-Westfalen,Early Stage Venture,$50M to $100M,251-500,3.0,Technology and Software,1,0,1,...,0,0,0,0,0,0,0,73000000.0,2,0.954702
8,8,Berlin,Early Stage Venture,$1M to $10M,51-100,2.0,Technology and Software,1,0,1,...,0,0,0,0,0,1,0,40196721.0,2,0.649737
9,9,Berlin,Seed,Less than $1M,101-250,3.0,Technology and Software,1,1,1,...,0,0,0,0,0,0,0,95000000.0,2,0.379408
13,13,Nordrhein-Westfalen,Early Stage Venture,$10M to $50M,101-250,1.0,Retail and E-commerce,1,1,0,...,0,0,0,0,0,1,0,116787884.0,2,0.923170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2169,2180,Baden-Wurttemberg,Seed,$1M to $10M,11-50,1.0,Technology and Software,1,1,1,...,0,0,0,0,0,0,0,27284.0,2,0.784431
2170,2181,Hessen,Seed,Less than $1M,11-50,1.0,Business Services,1,1,1,...,0,0,0,1,0,0,0,81868.0,2,0.649491
2173,2184,Berlin,Seed,$1M to $10M,11-50,1.0,Technology and Software,1,0,1,...,0,0,0,0,0,0,0,145515.0,2,0.877141
2174,2185,Hessen,Seed,$1M to $10M,11-50,2.0,Consumer Products,1,0,1,...,0,0,0,0,0,0,0,75000.0,2,0.406995


In [28]:
df.to_csv("../raw_data/predictions.csv")

In [31]:
model.save("dummy_model.keras")

In [32]:
new_df = pd.read_csv("../raw_data/new_df2.csv")

In [34]:
new_df["old_y"] = data["y"]

In [37]:
new_df.funding_status

0       Venture - Series Unknown
1                       Series B
2                       Series C
3                       Series C
4                       Series B
                  ...           
2184                        Seed
2185                        Seed
2186                        Seed
2187                        Seed
2188                         M&A
Name: funding_status, Length: 2189, dtype: object